In [1]:
from grewpy import Corpus, Request

path_in = "/home/ziane212/Téléchargements/out_reparsing_extraction_1/1314_AttiremensetJugies_extraction04.07.24.conllu"
path_tsv = '/home/ziane212/Téléchargements/out_reparsing_extraction/1314_output.tsv'
path_selected = '/home/ziane212/Téléchargements/out_reparsing_extraction/1314_selected_sents.tsv'

corpus = Corpus(path_in)

req_ccomp_noConj_subj = Request('X -[ccomp]-> Y; Y -[nsubj|expl]-> S').without('Y -[conj]-> Z; Y << Z; G [form = "«"]; X << G; G << Y; S [lemma="me"|"te"|"se"]')
res_ccomp_noConj_subj = corpus.search(req_ccomp_noConj_subj)

req_ccomp_conj_subj = Request('R -[ccomp]-> X; X -[conj]-> Y; X << Y; Y -[nsubj|expl]-> S').without('G [form = "«"]; R << G; G << Y; S [lemma="me"|"te"|"se"]')
res_ccomp_conj_subj = corpus.search(req_ccomp_conj_subj)

req_advcl_noConj_subj = Request('X -[advcl]-> Y; Y -[nsubj|expl]-> S').without('Y -[conj]-> Z; Y << Z; Y -[mark]-> M; M [upos=ADP]; S [lemma="me"|"te"|"se"]')
res_advcl_noConj_subj = corpus.search(req_advcl_noConj_subj)

req_advcl_conj_subj = Request('R -[advcl]-> X; X -[conj]-> Y; X << Y; Y -[nsubj|expl]-> S').without('Y -[mark]-> M; M [upos=ADP]; S [lemma="me"|"te"|"se"]')
res_advcl_conj_subj = corpus.search(req_advcl_conj_subj)

req_ccomp_noConj = Request('X -[ccomp]-> Y').without('Y -[conj]-> Z; Y << Z; G [form = "«"]; X << G; G << Y; Y -[nsubj|expl]-> S')
res_ccomp_noConj = corpus.search(req_ccomp_noConj)

req_ccomp_conj = Request('R -[ccomp]-> X; X -[conj]-> Y; X << Y').without('G [form = "«"]; R << G; G << Y; Y -[nsubj|expl]-> S')
res_ccomp_conj = corpus.search(req_ccomp_conj)

req_advcl_noConj = Request('X -[advcl]-> Y').without('Y -[conj]-> Z; Y << Z; Y -[mark]-> M; M [upos=ADP]; Y -[nsubj|expl]-> S')
res_advcl_noConj = corpus.search(req_advcl_noConj)

req_advcl_conj = Request('R -[advcl]-> X; X -[conj]-> Y; X << Y').without('Y -[mark]-> M; M [upos=ADP]; Y -[nsubj|expl]-> S')
res_advcl_conj = corpus.search(req_advcl_conj)

connected to port: 42379


In [ ]:
import conllu
import csv


def extract_sentences_with_results(path_in, results, clause_type, file_mode):
    # Lire le fichier conllu
    with open(path_in, 'r', encoding='utf-8') as f:
        data = f.read()

    text = path_in.split('/')[-1]
    
    # Parse the conllu data
    sentences = conllu.parse(data)
    
    # Créer un dictionnaire de phrases indexées par sent_id
    sent_dict = {s.metadata['sent_id']: s for s in sentences}

    # Ouvrir un fichier CSV pour écrire les résultats
    with open(path_tsv, file_mode, newline='', encoding='utf-8') as tsvfile:
        fieldnames = [
            'Text', 'Sent_ID', 'Clause type', 'Verb position', 'Notes', 
            'Expressed subject', 'Preposed subject', 'Impersonal', 
            'Sujet nominal avec article', 'Preposed nominal objet', 
            'Particle', 'Left context', 'Pivot', 'Right Context'
        ]
        writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
        
        if file_mode == 'w':
            # Écrire l'en-tête seulement si le fichier est ouvert en mode écriture
            writer.writeheader()
        
        # Parcourir les résultats
        for result in results[1:]:
            sent_id = result['sent_id']
            if sent_id in sent_dict:
                sentence = sent_dict[sent_id]
                
                y_id = int(result['matching']['nodes']['Y'])
                pivot_id = ''
                for token in sentence:
                    if token['head']==y_id and (token['deprel']=='aux' or token['deprel']=='aux:pass' or token['deprel']=='cop'):
                        pivot_id = token['id']
                        break
                # pivot_id = [token['id'] for token in sentence if token['head']==y_id and (token['deprel']=='aux' or token['deprel']=='aux:pass')][0]

                if pivot_id == '':
                    # Déterminer la position du verbe (Y)
                    y_token = next((token for token in sentence if token['id'] == y_id), None)
                    left_context = ' '.join([token['form'] for token in sentence[:y_id-1]])
                    pivot = y_token['form']
                    right_context = ' '.join([token['form'] for token in sentence[y_id:]])
                else:
                    y_token = next((token for token in sentence if token['id'] == pivot_id), None)
                    left_context = ' '.join([token['form'] for token in sentence[:pivot_id-1]])
                    pivot = y_token['form']
                    right_context = ' '.join([token['form'] for token in sentence[pivot_id:]])

                # Écrire la ligne dans le fichier CSV
                if '_subj' in clause_type:
                    writer.writerow({
                        'Text': text,
                        'Sent_ID': sent_id,
                        'Clause type': clause_type,
                        'Verb position': '',
                        'Notes': '',
                        'Expressed subject': 'Yes',
                        'Preposed subject': '',
                        'Impersonal': '',
                        'Sujet nominal avec article': '',
                        'Preposed nominal objet': '',
                        'Particle': '',
                        'Left context': left_context,
                        'Pivot': pivot,
                        'Right Context': right_context
                    })
                else:
                    writer.writerow({
                        'Text': text,
                        'Sent_ID': sent_id,
                        'Clause type': clause_type,
                        'Verb position': '',
                        'Notes': '',
                        'Expressed subject': '',
                        'Preposed subject': '',
                        'Impersonal': '',
                        'Sujet nominal avec article': '',
                        'Preposed nominal objet': '',
                        'Particle': '',
                        'Left context': left_context,
                        'Pivot': pivot,
                        'Right Context': right_context
                    })

# Exécuter la fonction avec le chemin du fichier et les résultats
extract_sentences_with_results(path_in, res_ccomp_noConj_subj, 'ccomp_noConj_subj', 'w')
extract_sentences_with_results(path_in, res_ccomp_noConj, 'ccomp_noConj', 'a')
extract_sentences_with_results(path_in, res_ccomp_conj_subj, 'ccomp_conj_subj', 'a')
extract_sentences_with_results(path_in, res_ccomp_conj, 'ccomp_conj', 'a')
extract_sentences_with_results(path_in, res_advcl_noConj_subj, 'advcl_noConj_subj', 'a')
extract_sentences_with_results(path_in, res_advcl_noConj, 'advcl_noConj', 'a')
extract_sentences_with_results(path_in, res_advcl_conj_subj, 'advcl_conj_subj', 'a')
extract_sentences_with_results(path_in, res_advcl_conj, 'advcl_conj', 'a')


In [ ]:
import pandas as pd

def remove_duplicate_lines(input_file, output_file):
    df = pd.read_csv(input_file, sep='\t')
    df['Clause type'] = df['Clause type'].str.replace('_subj', '', regex=False)
    
    # Valeur spécifique à conserver dans la colonne
    valeur_specifique = 'Yes'

    # Séparation des lignes avec la valeur spécifique dans la colonne
    df_specifique = df[df['Expressed subject'] == valeur_specifique]

    # Séparation des autres lignes
    df_autres = df[df['Expressed subject'] != valeur_specifique]

    matching_columns = ['Text', 'Sent_ID', 'Clause type', 'Left context', 'Pivot', 'Right Context']

    # Suppression des doublons dans les autres lignes en considérant uniquement les autres colonnes
    df_autres_dedup = df_autres.drop_duplicates(subset=matching_columns)

    # Combinaison des deux DataFrames
    df_final = pd.concat([df_specifique, df_autres_dedup]).drop_duplicates(subset=matching_columns, keep='first')

    # df = df.drop_duplicates()

    df_final.to_csv(output_file, sep='\t', index=False)

remove_duplicate_lines(path_tsv, path_tsv)


In [ ]:
import random

def select_phrases(input_file, output_file):
    # Définir le nombre de phrases à sélectionner pour chaque type de clause
    selection_counts = {
        'ccomp_noConj': 245,
        'ccomp_conj': 245,
        'advcl_noConj': 225,
        'advcl_conj': 175
    }
    
    # Lire le fichier TSV
    with open(input_file, 'r', encoding='utf-8') as tsvfile:
        reader = csv.DictReader(tsvfile, delimiter='\t')
        rows = list(reader)
    
    # Séparer les phrases par type de clause
    grouped_by_clause_type = {}
    for row in rows:
        clause_type = row['Clause type']
        if clause_type not in grouped_by_clause_type:
            grouped_by_clause_type[clause_type] = []
        grouped_by_clause_type[clause_type].append(row)
    
    selected_phrases = []
    
    for clause_type, phrases in grouped_by_clause_type.items():
        if clause_type not in selection_counts:
            continue
        
        num_phrases = selection_counts[clause_type]
        available_phrases = len(phrases)
        
        if available_phrases <= num_phrases:
            # Si moins de phrases disponibles que demandé, sélectionner toutes les phrases
            total_selected = phrases
        else:
            # Séparer les phrases courtes et longues
            short_phrases = [p for p in phrases if len(p['Left context'].split() + [p['Pivot']] + p['Right Context'].split()) < 30]
            long_phrases = [p for p in phrases if len(p['Left context'].split() + [p['Pivot']] + p['Right Context'].split()) >= 30]
            
            # Calculer le nombre de phrases courtes et longues nécessaires
            num_short_phrases = int(num_phrases * 0.7)
            num_long_phrases = num_phrases - num_short_phrases
            
            # Si moins de phrases courtes que nécessaire, ajuster
            if len(short_phrases) < num_short_phrases:
                num_short_phrases = len(short_phrases)
                num_long_phrases = num_phrases - num_short_phrases
            # Si moins de phrases longues que nécessaire, ajuster
            if len(long_phrases) < num_long_phrases:
                num_long_phrases = len(long_phrases)
                num_short_phrases = num_phrases - num_long_phrases            
            
            # Prendre les phrases courtes
            selected_short_phrases = random.sample(short_phrases, num_short_phrases)
            
            # Prendre les phrases longues
            selected_long_phrases = random.sample(long_phrases, num_long_phrases)
            
            total_selected = selected_short_phrases + selected_long_phrases
        
        for phrase in total_selected:
            phrase['Length'] = 'short' if len(phrase['Left context'].split() + [phrase['Pivot']] + phrase['Right Context'].split()) < 30 else 'long'
        
        selected_phrases.extend(total_selected)
    
    # Écrire les phrases sélectionnées dans un nouveau fichier TSV
    with open(output_file, 'w', newline='', encoding='utf-8') as tsvfile:
        fieldnames = [
            'Text', 'Sent_ID', 'Clause type', 'Verb position', 'Notes', 
            'Expressed subject', 'Preposed subject', 'Impersonal', 
            'Sujet nominal avec article', 'Preposed nominal objet', 
            'Particle', 'Left context', 'Pivot', 'Right Context', 'Length'
        ]
        writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
        writer.writeheader()
        
        for row in selected_phrases:
            writer.writerow(row)

select_phrases(path_tsv, 
               path_selected)